In [1]:
from scipy import stats
import pandas as pd
import numpy as np

In [4]:
def vol_difference_test(months_before, months_after, dataset):
    """ 
    This function analyzes daily returns in specified election and non-election periods,
    computes annualized volatilities for both, and performs Levene's test to assess 
    variance differences.

    Args:
        months_before (integer): the number of months before an election to start an election period
        months_after (integer): the number of months after an election to end an election period
        dataset (string): the filepath of the dataset
        
    Returns:
        annualized_election_volatility (float): the annualized volatility of election periods
        annualized_non_election_volatility (float): the annualized volatility of non-election periods
        lev_stat (float): the Levene statistic for the daily returns of the non-election periods and election periods
        p_value (float): the p-value from the Levene test
    """
    # Load historical data
    data = pd.read_csv(dataset)
    data['Date'] = pd.to_datetime(data['Date'])
    data.set_index('Date', inplace=True)
    data = data[(data.index >= '1974-01-01') & (data.index <= '2023-12-31')]

    # Calculate daily returns
    data['Daily Return'] = data['Adj Close'].pct_change()
    data = data.dropna(subset=['Daily Return'])

    # Separate data into election periods and non-election periods
    election_dates = [pd.Timestamp("1976-11-02"), pd.Timestamp("1980-11-04"), pd.Timestamp("1984-11-06"), pd.Timestamp("1988-11-08"), pd.Timestamp("1992-11-03"), pd.Timestamp("1996-11-05"), pd.Timestamp("2000-11-07"), pd.Timestamp("2004-11-02"), pd.Timestamp("2008-11-04"), pd.Timestamp("2012-11-06"), pd.Timestamp("2016-11-08"), pd.Timestamp("2020-11-03")]
    election_periods = [(day - pd.DateOffset(months=months_before), day + pd.DateOffset(months=months_after)) for day in election_dates]
    data['In Election Cycle'] = data.index.to_series().apply(
        lambda date: any(start <= date <= end for start, end in election_periods)
    )
    election_data = data[data['In Election Cycle']]
    non_election_data = data[~data['In Election Cycle']]

    # Calculate volatility
    election_volatility = election_data['Daily Return'].std()
    non_election_volatility = non_election_data['Daily Return'].std()

    # Annualize the volatility assuming 252 trading days per year
    annualized_election_volatility = election_volatility * np.sqrt(252)
    annualized_non_election_volatility = non_election_volatility * np.sqrt(252)

    # Levene's Test to see if the variances of Daily Returns are equal
    lev_stat, p_value = stats.levene(election_data['Daily Return'], non_election_data['Daily Return'])

    return annualized_election_volatility, annualized_non_election_volatility, lev_stat, p_value



In [5]:
for x in range(1,13):
    for y in range(1,13):
        _, _, _, p_value = vol_difference_test(x, y, "GSPC.csv")
        print("Election Period: " + str(x) + " months before, " + str(y) + " months after. P-value: " + str(p_value))

Election Period: 1 months before, 1 months after. P-value: 5.25508238558292e-08
Election Period: 1 months before, 2 months after. P-value: 1.7662413134182485e-05
Election Period: 1 months before, 3 months after. P-value: 0.011230152196265589
Election Period: 1 months before, 4 months after. P-value: 0.028811292401368413
Election Period: 1 months before, 5 months after. P-value: 0.011855463374636008
Election Period: 1 months before, 6 months after. P-value: 0.046066581906858986
Election Period: 1 months before, 7 months after. P-value: 0.3163652722682728
Election Period: 1 months before, 8 months after. P-value: 0.8454993467440793
Election Period: 1 months before, 9 months after. P-value: 0.09208914114946472
Election Period: 1 months before, 10 months after. P-value: 0.004893921549387352
Election Period: 1 months before, 11 months after. P-value: 0.000919656448370338
Election Period: 1 months before, 12 months after. P-value: 0.00042874496813348267
Election Period: 2 months before, 1 mo